<a href="https://colab.research.google.com/github/whoamiwhress/blank/blob/main/minecraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minecraft on Colab  
For those people who run 💩 on Google Colaboratory.  
Because why NOT turn a machine learning notebook into a literal block party. 🎉  
🧱✨ Powered by ngrok, chaos, and the tears of sysadmins.  

In [ ]:
#@title 🚀 Minecraft Server on Colab (Main Launch Cell v2)
#@markdown ## Minecraft on Colab
#@markdown _Blocky chaos in the cloud, powered by Google and Gen Z._
#@markdown ---
#@markdown 🔄 **Choose Version Type**
release_type = "release" #@param ["release", "snapshot"]
#@markdown 🔐 **Your ngrok Authtoken** (Optional)
authtoken = "" #@param {type:"string"}

import subprocess
import sys
import re
import os

# 🧙‍♂️ pip auto install
def auto_install(package):
    try:
        __import__(package)
    except ImportError:
        print(f"[⚠️] Missing '{package}', trying to install...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except subprocess.CalledProcessError:
            print(f"[💥] Standard pip install failed. Trying with --break-system-packages...")
            try:
                subprocess.check_call([
                    sys.executable, "-m", "pip", "install", "--break-system-packages", package
                ])
            except subprocess.CalledProcessError:
                print(f"[💀] Even with break-system-packages, install failed for {package}. You're cursed.")
                sys.exit(1)

for pkg in ["httpx", "pyngrok"]:
    auto_install(pkg)

import httpx
from pyngrok import ngrok

# 🔥 Get latest version
def get_latest(release_type="release"):
    url = "https://launchermeta.mojang.com/mc/game/version_manifest_v2.json"
    try:
        res = httpx.get(url, timeout=10.0)
        res.raise_for_status()
        data = res.json()
        if release_type not in ["release", "snapshot"]:
            print(f"[WARN] Invalid release_type '{release_type}', defaulting to 'release'")
            release_type = "release"
        latest_version = data["latest"][release_type]
        print(f"[🆕] Latest {release_type}: {latest_version}")
        return latest_version
    except Exception as e:
        print(f"[X] Failed to fetch latest version: {e}")
        return None

# 📰 Get article URL
def get_article_url(version):
    if re.match(r"..w..+", version):
        return f"https://www.minecraft.net/en-us/article/minecraft-snapshot-{version}"
    elif "pre" in version:
        slug = version.replace('.', '-').replace(' ', '-').lower()
        return f"https://www.minecraft.net/en-us/article/minecraft-{slug}"
    else:
        slug = version.replace('.', '-')
        return f"https://www.minecraft.net/en-us/article/minecraft-java-edition-{slug}"

# 🔍 Find the JAR link
def fetch_server_jar(version):
    url = get_article_url(version)
    print(f"[🔗] Fetching article: {url}")
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)",
        "Accept": "text/html",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive",
    }
    try:
        with httpx.Client(http2=True, headers=headers, timeout=15.0) as client:
            res = client.get(url)
            if "Minecraft server jar" in res.text:
                match = re.search(r'https://piston-data\.mojang\.com/v1/objects/[a-f0-9]+/server\.jar', res.text)
                if match:
                    jar_url = match.group()
                    print(f"[🎯] Server JAR found: {jar_url}")
                    return jar_url
    except httpx.RequestError as e:
        print(f"[X] HTTPX failed with: {e}")
    return None

# 📦 Download the JAR
def download_jar(jar_url, dest_path="server.jar"):
    try:
        print(f"[⬇️] Downloading server JAR to {dest_path}...")
        res = httpx.get(jar_url)
        with open(dest_path, "wb") as f:
            f.write(res.content)
        print(f"[✅] Download complete.")
    except Exception as e:
        print(f"[X] Download failed: {e}")

# 🎮 Launch MC server
def start_server():
    if not os.path.exists("eula.txt"):
        with open("eula.txt", "w") as f:
            f.write("eula=true\n")
    print("[🧱] Starting Minecraft server...")
    subprocess.Popen(["java", "-Xms1G", "-Xmx1G", "-jar", "server.jar", "nogui"])

# 🌐 Start ngrok tunnel
def start_ngrok(authtoken=None):
    if authtoken:
        ngrok.set_auth_token(authtoken)
    tunnel = ngrok.connect(25565, "tcp")
    print(f"[🌍] ngrok tunnel is LIVE → {tunnel.public_url}")
    print("⚠️ Use this address in your Minecraft client!")

# 🧠 Main brain
version = get_latest(release_type)
print(f"[🆕] Using version: {version}")
jar_url = fetch_server_jar(version)

if jar_url:
    download_jar(jar_url)
    start_server()
    if authtoken:
        start_ngrok(authtoken)
    else:
        print("[🚫] Skipping ngrok — no authtoken provided.")
else:
    print("[❌] Couldn't find server JAR, sorry bruh.")
